In [12]:
import xarray as xr
import numpy as np 
import glob 
import os 
import pickle

os.chdir("/home/bsc32/bsc32263/git/one_pass")
from one_pass.opa import *
from one_pass.opa import Opa

In [24]:

#filePathData = "/esarchive/scratch/alacima/projects/destination_earth/icon/*.nc"

# imagine this is the gsv request ## 
file_path_data = "/home/bsc32/bsc32263/git/one_pass/uas_10_months.nc"
fileList = glob.glob(file_path_data) 
fileList.sort() 
data = xr.open_dataset(fileList[0])  # , chunks = 'auto') # open dataset
data = data.astype(np.float64)
data


<xarray.Dataset>
Dimensions:   (time: 6960, lat: 100, lon: 100)
Coordinates:
  * time      (time) datetime64[ns] 2020-05-01T00:15:00 ... 2021-09-30T23:15:00
  * lon       (lon) float64 -174.9 -174.9 -174.8 -174.8 ... -170.1 -170.0 -170.0
  * lat       (lat) float64 -85.0 -84.95 -84.9 -84.85 ... -80.15 -80.1 -80.05
    height_2  float64 ...
Data variables:
    uas       (time, lat, lon) float64 -0.9785 -1.054 ... 0.008314 0.008062

In [3]:
n_start = 2*30*24 
n_data = n_start + 3*30*24 + 2*24 

pass_dic = {"stat": "std",
"stat_freq": "3monthly",
"output_freq": "3monthly",
"time_step": 60,
"variable": "uas",
"save": True,
"checkpoint": True,
"checkpoint_file": "/home/bsc32/bsc32263/git/data/checkpoint_in_std_3hourly.nc",
"out_file": "/home/bsc32/bsc32263/git/data/"}

#daily_std = Opa(pass_dic)

for i in range(n_start, n_data, 1): 

    ds = data.isel(time=slice(i,i+1)) # extract moving window of tas variable
    daily_std = Opa(pass_dic)
    dm = daily_std.compute(ds) # this can overwrite the checkpoint file if is sees that it's the first of each stat 

dm

starting_save
finished saving
starting_save
finished saving


<xarray.Dataset>
Dimensions:   (time: 7, lat: 100, lon: 100)
Coordinates:
    height_2  float64 10.0
  * time      (time) datetime64[ns] 2020-05-02T03:00:00 ... 2020-05-02T21:00:00
  * lon       (lon) float64 -174.9 -174.9 -174.8 -174.8 ... -170.1 -170.0 -170.0
  * lat       (lat) float64 -85.0 -84.95 -84.9 -84.85 ... -80.15 -80.1 -80.05
Data variables:
    uas       (time, lat, lon) float64 1.443 1.375 1.332 ... 0.8908 0.8995 0.898
Attributes:
    OPA:      3hourly std calculated using one-pass algorithm

In [7]:
dm.uas

<xarray.DataArray 'uas' (time: 7, lat: 100, lon: 100)>
array([[[1.44349757, 1.37514059, 1.33199554, ..., 0.45843832,
         0.45880783, 0.57007175],
        [0.36344916, 0.37512033, 0.48164039, ..., 0.16707988,
         0.15929183, 0.15847606],
        [0.36915206, 0.52369567, 1.20225443, ..., 0.3473276 ,
         0.33387824, 0.31024312],
        ...,
        [0.38040807, 0.41982143, 0.42311188, ..., 0.31612836,
         0.35691232, 0.35917223],
        [0.49139599, 0.55597037, 0.56214756, ..., 0.20030558,
         0.20206238, 0.20344699],
        [0.62378851, 0.64613208, 0.65561781, ..., 0.11608167,
         0.11471499, 0.11688665]],

       [[0.2404671 , 0.21793848, 0.19636221, ..., 0.36733574,
         0.36840331, 0.38270012],
        [0.11477862, 0.22069731, 0.23043802, ..., 0.49272057,
         0.50031614, 0.51102696],
        [0.24864227, 0.21731464, 0.02354116, ..., 0.630828  ,
         0.6260546 , 0.57714463],
...
        [0.75730801, 0.82214794, 0.83056866, ..., 0.71790128,
         0.50335519, 0.496845  ],
        [0.59396923, 0.62840855, 0.63208014, ..., 0.76922131,
         0.74553381, 0.72648365],
        [0.33257682, 0.39194954, 0.40078795, ..., 1.03848004,
         1.02517929, 1.01372419]],

       [[0.35960373, 0.31672466, 0.27275545, ..., 0.37042313,
         0.4156387 , 0.52693434],
        [0.24184359, 0.31510949, 0.33566591, ..., 0.81409256,
         0.83244862, 0.84609038],
        [0.33151344, 0.31283494, 0.34763827, ..., 0.78014644,
         0.77554984, 0.688816  ],
        ...,
        [0.49585727, 0.57431838, 0.59208272, ..., 0.88639939,
         0.91256127, 0.91582039],
        [0.4622166 , 0.53841107, 0.54470767, ..., 0.93609009,
         0.94272045, 0.94837351],
        [0.160814  , 0.23430309, 0.24595596, ..., 0.8908137 ,
         0.89951518, 0.89799452]]])
Coordinates:
    height_2  float64 10.0
  * time      (time) datetime64[ns] 2020-05-02T03:00:00 ... 2020-05-02T21:00:00
  * lon       (lon) float64 -174.9 -174.9 -174.8 -174.8 ... -170.1 -170.0 -170.0
  * lat       (lat) float64 -85.0 -84.95 -84.9 -84.85 ... -80.15 -80.1 -80.05
Attributes:
    standard_name:  uas
    long_name:      zonal wind in 10m
    units:          m s-1
    param:          2.2.0
    cell_methods:   time: mean
    OPA:            3hourly std calculated using one-pass algorithm

In [11]:
dm1 = getattr(dm, pass_dic["variable"])


In [42]:
time_stamp_list = data.time.data[31*24] # assuming that incoming data has a time dimension
time_stamp_pandas = [pd.to_datetime(time_stamp_list)]
time_stamp = time_stamp_pandas[0] # converting to a pandas datetime to calculate if it's the first


time_stamp_min = time_stamp.minute
time_stamp_hour = time_stamp.hour*60 # converting to minutes -
time_stamp_day = (time_stamp.day-1)*24*60 # .day corresponds to day of the month

time_stamp_month = time_stamp.month
time_stamp_month_tot = time_stamp_month* time_stamp.days_in_month*24*60
    
if(np.mod(time_stamp_month, 6) != 0):
    time_stamp_tot = time_stamp_min + time_stamp_hour + time_stamp_day + time_stamp_month_tot
else:
    time_stamp_tot = time_stamp_min + time_stamp_hour + time_stamp_day

time_stamp

Timestamp('2020-06-01 00:15:00')

In [82]:
n_start = 2*30*24 
n_data = n_start + 3*30*24 + 2*24
data.time.data[n_start] 
data.time.data[n_data] 


numpy.datetime64('2021-05-01T00:15:00.000000000')

In [7]:
u_all = data.uas
ds = u_all.isel(time=slice(0,1)) 
count = 1 
weight = 1
ds_size = ds.tail(time = 1)
value = np.zeros_like(ds_size)
mean_cum = value + weight*(ds - value) / (count) # updating mean with one-pass algorithm
mean_cum
ds

<xarray.DataArray 'uas' (time: 1, lat: 100, lon: 100)>
array([[[-0.978494, -1.053661, ..., -3.308914, -3.5594  ],
        [-5.918924, -9.31018 , ..., -8.41738 , -8.376527],
        ...,
        [-0.496185, -0.498889, ...,  0.639367,  0.626193],
        [-0.393446, -0.408896, ...,  0.333626,  0.306882]]], dtype=float32)
Coordinates:
  * time      (time) datetime64[ns] 2020-05-01T00:15:00
  * lon       (lon) float64 -174.9 -174.9 -174.8 -174.8 ... -170.1 -170.0 -170.0
  * lat       (lat) float64 -85.0 -84.95 -84.9 -84.85 ... -80.15 -80.1 -80.05
    height_2  float64 ...
Attributes:
    standard_name:  uas
    long_name:      zonal wind in 10m
    units:          m s-1
    param:          2.2.0
    cell_methods:   time: mean

In [25]:
pass_dic = {"stat": "std",
"stat_freq": "monthly",
"output_freq": "monthly",
"time_step": 60,
"variable": "uas",
"save": False,
"checkpoint": True,
"checkpoint_in_file": "/home/bsc32/bsc32263/git/data/checkpoint_in_std_monthly.nc",
"out_file": "/home/bsc32/bsc32263/git/data/"}

n_data = 31*24
u_all = data.uas 

for i in range(0, n_data, 1): 

    ds = u_all.isel(time=slice(i,i+1)) 
    monthly_std = Opa(pass_dic)
    dm = monthly_std.compute(ds)

## run two pass std

ds = data.uas.isel(time=slice(0, n_data))
axNum = ds.get_axis_num('time')
np_std = np.std(ds, axis = axNum, dtype = np.float64, ddof =1, keepdims = True)

dec_placelace = 1e-12
message = "OPA mean and numpy mean not equal to 7 dp"

np.allclose(dm.uas, np_std, rtol= dec_placelace, atol = 0)



True

In [26]:
dm.uas

<xarray.DataArray 'uas' (time: 1, lat: 100, lon: 100)>
array([[[1.77962062, 1.87152545, 1.97521208, ..., 1.72755751,
         1.76490226, 1.88145462],
        [2.61439035, 3.57342403, 3.76666732, ..., 2.91976098,
         2.90421949, 2.90065089],
        [4.33290441, 4.55184625, 5.61446787, ..., 3.59867964,
         3.56405937, 3.37528397],
        ...,
        [2.9620461 , 2.97012401, 2.97098405, ..., 3.42874544,
         3.43475642, 3.43611163],
        [2.95167706, 2.95823016, 2.95872582, ..., 3.41538507,
         3.41640584, 3.41702357],
        [2.95944663, 2.9627381 , 2.96338745, ..., 3.3936618 ,
         3.39385826, 3.39397332]]])
Coordinates:
    height_2  float64 10.0
  * time      (time) datetime64[ns] 2020-05-01
  * lon       (lon) float64 -174.9 -174.9 -174.8 -174.8 ... -170.1 -170.0 -170.0
  * lat       (lat) float64 -85.0 -84.95 -84.9 -84.85 ... -80.15 -80.1 -80.05
Attributes:
    standard_name:  uas
    long_name:      zonal wind in 10m
    units:          m s-1
    param:          2.2.0
    cell_methods:   time: mean
    OPA:            monthly std calculated using one-pass algorithm

In [27]:
np_std

<xarray.DataArray 'uas' (time: 1, lat: 100, lon: 100)>
array([[[1.77962062, 1.87152545, 1.97521208, ..., 1.72755751,
         1.76490226, 1.88145462],
        [2.61439035, 3.57342403, 3.76666732, ..., 2.91976098,
         2.90421949, 2.90065089],
        [4.33290441, 4.55184625, 5.61446787, ..., 3.59867964,
         3.56405937, 3.37528397],
        ...,
        [2.9620461 , 2.97012401, 2.97098405, ..., 3.42874544,
         3.43475642, 3.43611163],
        [2.95167706, 2.95823016, 2.95872582, ..., 3.41538507,
         3.41640584, 3.41702357],
        [2.95944663, 2.9627381 , 2.96338745, ..., 3.3936618 ,
         3.39385826, 3.39397332]]])
Coordinates:
  * lon       (lon) float64 -174.9 -174.9 -174.8 -174.8 ... -170.1 -170.0 -170.0
  * lat       (lat) float64 -85.0 -84.95 -84.9 -84.85 ... -80.15 -80.1 -80.05
    height_2  float64 10.0
Dimensions without coordinates: time

In [16]:

n_data = 24

pass_dic = {"stat": "mean",
"stat_freq": "daily",
"output_freq": "daily",
"time_step": 60,
"variable": "uas",
"save": False,
"checkpoint": False,
"checkpoint_in_file": "/home/bsc32/bsc32263/git/data/checkpoint_in_mean_daily.nc",
"out_file": "/home/bsc32/bsc32263/git/data/"}

data = data.astype(np.float64)
daily_mean = Opa(pass_dic)

for i in range(0, n_data, 1): 
    ds1 = data.isel(time=slice(i,i+1)) # extract moving window
    dm = daily_mean.compute(ds1)

## run two pass mean 

ds = data.uas.isel(time=slice(0, n_data))
axNum = ds.get_axis_num('time')
np_mean = np.mean(ds, axis = axNum, dtype = np.float64, keepdims = True)

dec_placelace = 1e-12
message = "OPA mean and numpy mean not equal to 7 dp"

np.allclose(dm.uas, np_mean, rtol= dec_placelace, atol = 0)



True

In [17]:
dm.uas.values

array([[[ 0.06434895,  0.01239685, -0.03355959, ..., -1.77226271,
         -1.81505311, -1.2741398 ],
        [-1.08329271, -2.38118367, -2.51244453, ..., -6.42560913,
         -6.22782701, -6.08368877],
        [-3.75257562, -4.09614989, -5.60846062, ..., -9.39340544,
         -9.25492684, -8.52982157],
        ...,
        [-1.66970807, -1.70690147, -1.70906749, ..., -2.02470259,
         -2.03432343, -2.03686662],
        [-1.64076491, -1.66352987, -1.66804871, ..., -2.11496041,
         -2.11864636, -2.12220103],
        [-1.59706481, -1.62157481, -1.6242384 , ..., -2.21520563,
         -2.21440249, -2.22324883]]])

In [18]:
np_mean.values

array([[[ 0.06434895,  0.01239685, -0.03355959, ..., -1.77226271,
         -1.81505311, -1.2741398 ],
        [-1.08329271, -2.38118367, -2.51244453, ..., -6.42560913,
         -6.22782701, -6.08368877],
        [-3.75257562, -4.09614989, -5.60846062, ..., -9.39340544,
         -9.25492684, -8.52982157],
        ...,
        [-1.66970807, -1.70690147, -1.70906749, ..., -2.02470259,
         -2.03432343, -2.03686662],
        [-1.64076491, -1.66352987, -1.66804871, ..., -2.11496041,
         -2.11864636, -2.12220103],
        [-1.59706481, -1.62157481, -1.6242384 , ..., -2.21520563,
         -2.21440249, -2.22324883]]])

In [35]:

message = "OPA mean and numpy mean not equal to " + str(dec_placelace) + " dp"
message


'OPA mean and numpy mean not equal to 1e-12 dp'